In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xgboost as xgb
import lightgbm as lgb

pd.set_option('display.max_columns', 81)

In [38]:
# trainデータを取得、NaN値などの状況を確認
dataset = pd.read_csv('data/train.csv', sep=',')
dataset_test = pd.read_csv('data/test.csv', sep=',')
train_eol = dataset.shape[0]
dataset = pd.concat([dataset, dataset_test], axis=0)
dataset.iloc[0:train_eol,:]
dataset.iloc[train_eol:,:]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,NaN
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,NaN
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,NaN
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal,NaN
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960.0,Unf,2.0,576.0,TA,TA,Y,474,0,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,

In [2]:
## NaNデータの処理
# 以下のカラムリストについては、NAのものは"None"に置き換えることで欠損値を補完する

def feature_engineering(df_train, df_test):
    # Trainデータの末尾indexを取得して、trainとtestの分割に使用
    # train側のデータを2つ削除している
    train_eol = df_train.shape[0]
    dataset = pd.concat([df_train, df_test], axis=0)

    Id = dataset['Id']
    dataset = dataset.drop(['Id'], axis=1)
    
    # ************************************************************* NaN処理 *************************************************************
    convert_nan_to_other_list = ['MSSubClass', 'Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                                'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
                                'PoolQC', 'Fence', 'MiscFeature']
    convert_nan_to_other_list = dict.fromkeys(convert_nan_to_other_list, 'None')
    dataset = dataset.fillna(convert_nan_to_other_list)

    # LotFrontageのNAは、各近接地における中央値で置換
    dataset['LotFrontage'] = dataset['LotFrontage'].fillna(dataset.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median())))
    # Functionalについてはdescriptionに"Assume typical unless deductions are warranted"とある
    dataset['Functional'] = dataset['Functional'].fillna('Typ')

    # ************************* 0でfillna *************************
    for x in ('GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'MasVnrArea'):
        dataset[x] = dataset[x].fillna(0)

    # ************************* 最頻値でfillna *************************
    for x in ('MSZoning', 'Electrical', 'KitchenQual', 'Exterior1st', 'Exterior2nd', 'SaleType'):
        dataset[x] = dataset[x].fillna(dataset[x].mode()[0])

    dataset = dataset.drop(['Utilities'],axis=1)

    if dataset.isnull().sum().sum() == 0:
        print('Dataset doesnt have NaN')
    # ************************************************************* NaN完了 *************************************************************


    # *********************************************************** Feature Engineering *************************************************************
    categorical = ['MSSubClass', 'OverallQual','OverallCond', 'YrSold', 'MoSold']
    for x in categorical:
        dataset[x] = dataset[x].astype(str)

    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    # データセットが情報を表す順序で並んでいる"だろう"ものについて、label encoding
    label_encoding = ['MSSubClass', 'Street', 'Alley', 'LotShape', 'LandSlope', 'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 
                    'BsmtQual', 'BsmtCond', 'BsmtExposure','BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir', 'KitchenQual',
                    'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'MoSold', 'YrSold']
    for x in label_encoding:
        dataset[x] = label_encoder.fit_transform(dataset[x].values)

    dataset['TotalSF'] = dataset['TotalBsmtSF'] + dataset['1stFlrSF'] + dataset['2ndFlrSF']
    
    # 数値尺度のカラムを取得
    numeric_cols = dataset.dtypes[dataset.dtypes != 'object'].index
    skew = dataset.loc[:,numeric_cols].apply(lambda x: x.skew()).sort_values(ascending=False)
    # skewが中央にないものは、log変換
    skewness = skew[abs(skew) > 0.75].index
    dataset[skewness] = np.log1p(dataset[skewness])

    onehot_encoding = dataset.dtypes[dataset.dtypes == 'object'].index
    dataset = pd.get_dummies(dataset, columns=onehot_encoding, drop_first=True)
    # ********************************************************* End of Feature Engineering **********************************************************
    
    dataset['Id'] = Id
    train = dataset.iloc[0:train_eol,:]
    
    # trainデータから明らかな外れ値についてはデータから削除
    outliers = [ 524, 1299]
    for x in outliers:
        train = train.drop(train[train['Id'] == x].index)
    
    test = dataset.iloc[train_eol:,:]
    
    return train, test

dataset = pd.read_csv('data/train.csv', sep=',')
dataset_test = pd.read_csv('data/test.csv', sep=',')

train, test = feature_engineering(dataset, dataset_test)

def convert_model_data(data):
    Id = data['Id'].values
    X = data.drop(['Id', 'SalePrice'], axis=1).values
    y = data['SalePrice'].values
    return Id, X, y
Id, X, y = convert_model_data(train)

In [6]:
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_val_score

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

{'bagging_fraction': 0.1,
 'learning_rate': 0.1,
 'max_depth': 10,
 'n_estimators': 200,
 'num_leaves': 100}

def cross_val(model):
    score = cross_val_score(model, X, y, scoring='r2', cv=kfold)
    print(f'{type(model).__name__} score = : {score.mean():.8f}')

linear = make_pipeline(RobustScaler(), LinearRegression())
lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.0005, random_state=0))
elastic = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=0)) 
kernel = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
lgb_regressor = lgb.LGBMRegressor(objective='regression', num_leaves=5, learning_rate=0.05, n_estimators=720, bagging_fraction=0.8)
rndm_regressor = RandomForestRegressor(n_estimators = 100, criterion='mae', random_state=0, bootstrap=True)
gboost = GradientBoostingRegressor(n_estimators = 3000, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state=0)
xgb_model = xgb.XGBRegressor(colsample_bytree=0.463, gamma=0.0468, learning_rate=0.05, max_depth=3, min_child_weight=1.7817, n_estimators=2200, reg_alpha=0.4640, reg_lambda=0.8571, subsample=0.5213, silent=1, random_state=0, nthread=-1)

for x in (linear, lasso, elastic, kernel, lgb_regressor, gboost, xgb_model, rndm_regressor):
    cross_val(x)

Pipeline score = : 0.89309526
Pipeline score = : 0.91229383
Pipeline score = : 0.91237290
KernelRidge score = : 0.85600888
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1
LGBMRegressor score = : 0.89799647
GradientBoostingRegressor score = : 0.91395769
XGBRegressor score = : 0.91032509
RandomForestRegressor score = : 0.88168091


In [98]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.metrics import r2_score
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
    
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]

        for model in self.models_:
            model.fit(X, y)
            
        return self
    
    def predict(self, X):
        predictions = np.column_stack(
            [model.predict(X) for model in self.models_]
            )
        return np.mean(predictions, axis=1)

averaged_models = AveragingModels(models = (linear, lasso, elastic, gboost))
averaged_models.fit(X, y)

# テストデータを取得
Id, X_test, y_test = convert_model_data(test)
# 予想してみる
y_pred = averaged_models.predict(X_test)
y_pred = np.exp(y_pred)
output = pd.DataFrame({'Id': Id, 'SalePrice': y_pred})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [5]:
# grid search
from sklearn.model_selection import GridSearchCV

'''
rndm_grid_parameters = [
    {'n_estimators': [5, 10, 100, 500, 1000], 
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 5, 10],
    'max_depth': [3, 5, 7, 10, 15 None]
    'bootstrap': [True, False],
    'max_features': ['sqrt', 'log2']
    }
]

grid_search = GridSearchCV(RandomForestRegressor(), rndm_grid_parameters, cv=kfold, scoring='r2', n_jobs = -1, verbose=10)
grid_search.fit(X, y)
grid_search.best_params_
'''
xg_param_grid = {"max_depth": [ 3, 6, 10,25],
              "learning_rate" : [0.0001,0.001,0.01],
              "min_child_weight" : [1,3,6],
              "n_estimators": [100,200,300],
              "subsample": [0.5,0.75,0.9],
              "gamma":[0,0.1,0.2],
              "eta": [0.3,0.15,0.10]
             }
#grid_search = GridSearchCV(xgb.XGBRegressor(), xg_param_grid, cv=kfold, scoring='r2', n_jobs=-1, verbose=10)
#grid_search.fit(X, y)
#grid_search.best_params_

lgb_param_grid = {"max_depth": [10, 25, 50, 75],
              "learning_rate" : [0.001,0.01,0.05,0.1],
              "num_leaves": [100,300,900,1200],
              "n_estimators": [100,200,500],
              "bagging_fraction": [0.1, 0.5, 0.8, 1.0]
             }
grid_search = GridSearchCV(lgb.LGBMRegressor(), lgb_param_grid, cv=kfold, scoring='r2', n_jobs=-1, verbose=10)
grid_search.fit(X, y)
grid_search.best_params_

Fitting 5 folds for each of 768 candidates, totalling 3840 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   31.3s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   35.8s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   46.6s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:   52.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   57.1s
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed: 

{'bagging_fraction': 0.1,
 'learning_rate': 0.1,
 'max_depth': 10,
 'n_estimators': 200,
 'num_leaves': 100}